<a href="https://colab.research.google.com/github/sualeh/introduction-to-chatgpt-api/blob/main/chatgpt-function-calls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------

> **How to Run This Notebook**

To get started, create an Open AI API account, set up billing, and generate and API key at https://platform.openai.com/. If you are running the notebook locally in Visual Studio Code or other IDE, create a file called `.env`, and add a line `OPENAI_API_KEY=<your-openai-api-key>`. This key will be read by the `load_dotenv` library.

Otherwise, if you are running in Google Colab, create a secret called `OPENAI_API_KEY` and set it to the value of your OpenAI API key.

Run the code below to read the key.


In [ ]:
%pip install python-dotenv

from os import environ as env
from dotenv import load_dotenv

# Load key from an environmental variable called "OPENAI_API_KEY"
# Use python-dotenv https://pypi.org/project/python-dotenv/
# And take environment variables from .env
load_dotenv()
try:
  # Attempt to read OPENAI_API_KEY from a Google Colab secret
  from google.colab import userdata
  env['OPENAI_API_KEY'] = env.get('OPENAI_API_KEY', userdata.get('OPENAI_API_KEY'))
except ModuleNotFoundError:
  print("Not running in Google Colab")
  # No action - rely on the OPENAI_API_KEY environmental variable



----------

# Using Function Calls in ChatGPT API


## Getting Started

In [ ]:
%pip install openai

from openai import OpenAI

# Create Open AI client to use ChatGPT
client = OpenAI()

## Making Function Calls

ChatGPT does not have access to the capabilities of your software system. If you can describe the capabilities of your system as if they were function calls, you can use the power of ChatGPT to understand a request from the user, and tell you which capability will fulfill the request.

Let us say that your system is able to get the weather, if you provide it with a location. You can express this system capability as a function like the one below, and describe it to ChatGPT. ChatGPT will determine if the function can satisfy the user prompt, and will determine the arguments to use.

In [ ]:
def get_current_weather(location: str, unit="fahrenheit"):
    """
    Get the current weather in a given location.

    Parameters:
        location (str): Location that includes both the city and state, e.g. San Francisco, CA
        unit (str, optional): The unit of temperature. Defaults to "fahrenheit".

    Returns:
        str: A JSON string containing the weather information. The JSON string includes the location, temperature, unit of temperature, and forecast.

    Example:
        >>> get_current_weather("New York")
        '{"location": "New York", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'
    """
    # THIS NOT A REAL FUNCTION, BUT A CAPABILITY THAT YOUR SYSTEM CAN EXECUTE

If you describe this function and it's parameters to ChatGPT, ChatGPT will determine the intent to use this function.

In [ ]:
user_prompt = "What's the weather like in Boston?"

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": user_prompt}],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Location that includes both the city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit of temperature",
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call="auto",
)

We name and describe the function, then describe the parameters that to pass to this function. ChatGPT relies on this description to help identify whether to call the function, and what the arguments should be.

If we specify `function_call="auto"`, ChatGPT will try to fulfill the function parameters. If set to `none` you can force no function to be detected.


Here is the prompt returned:

In [ ]:
reply_content = chat_completion.choices[0].message.function_call
reply_content

Notice that there is no content, but instead a function call, with extracted parameters in a JSON object. It is your responsibility to call the function if you would like to do that.

## References

- Based on [Sentdex/ChatGPT-API-Basics](https://github.com/Sentdex/ChatGPT-API-Basics)
- See [ChatGPT API in Python by sentdex](https://www.youtube.com/watch?v=c-g6epk3fFE)